In [1]:
# Instalação dos Pacotes

!pip install pyspark
!pip install findspark

In [2]:
#importando o findSpark

import findspark
findspark.init()

In [3]:
#importando algumas bibliotecas e funções

import re
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [4]:
#Regex utilizados

In [126]:
# Criar o contexto do spark
sc = SparkContext()

# Instancia o criador de sessao do spark
spark = (SparkSession.builder
                     .master("local[*]")
                     .appName("Aceleração PySpark - Semana 3 - Capgemini"))

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at C:\Users\brsantan\AppData\Local\Temp/ipykernel_21744/1672453917.py:2 

In [127]:
#Schema dos datasets

schema_airports = StructType([
    StructField("faa",  StringType(),  True),
    StructField("name", StringType(),  True),
    StructField("lat",  FloatType(),   True),
    StructField("lon",  FloatType(),   True),
    StructField("alt",  IntegerType(), True),
    StructField("tz",   IntegerType(), True),
    StructField("dst",  StringType(),  True)
])

schema_planes = StructType([
    StructField("tailnum",      StringType(),  True),
    StructField("year",         IntegerType(), True),
    StructField("type",         StringType(),  True),
    StructField("manufacturer", StringType(),  True),
    StructField("model",        StringType(),  True),
    StructField("engines",      IntegerType(), True),
    StructField("seats",        IntegerType(), True),
    StructField("speed",        IntegerType(), True),
    StructField("engine",       StringType(),  True)
])

schema_flights = StructType([
    StructField("year",      IntegerType(), True),
    StructField("month",     IntegerType(), True),
    StructField("day",       IntegerType(), True),
    StructField("dep_time",  StringType(),  True),
    StructField("dep_delay", IntegerType(), True),
    StructField("arr_time",  StringType(),  True),
    StructField("arr_delay", IntegerType(), True),
    StructField("carrier",   StringType(),  True),
    StructField("tailnum",   StringType(),  True),
    StructField("flight",    StringType(),  True),
    StructField("origin",    StringType(),  True),
    StructField("dest",      StringType(),  True),
    StructField("air_time",  IntegerType(), True),
    StructField("distance",  IntegerType(), True),
    StructField("hour",      IntegerType(), True),
    StructField("minute",    IntegerType(), True),
])



In [128]:
#Criando os Data Frame

df_airports = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_airports)
                  .load("./datasets/aviao/airports.csv"))

df_planes = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_planes)
                  .load("./datasets/aviao/planes.csv"))

df_flights = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_flights)
                  .load("./datasets/aviao/flights.csv"))

In [129]:
df_flights = df_flights[['year', 'month', 'day','hour', 'minute','dep_time','arr_time', 'dep_delay', 'arr_delay', 'carrier', 'tailnum', 'flight',
        'origin','dest', 'air_time', 'distance']]

In [130]:
#Visões Temporarias

df_airports.createOrReplaceTempView('airports')

df_planes.createOrReplaceTempView('planes')

df_flights.createOrReplaceTempView('flights')

In [131]:
df_airports.show(5)

+---+--------------------+---------+---------+----+---+---+
|faa|                name|      lat|      lon| alt| tz|dst|
+---+--------------------+---------+---------+----+---+---+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|
|06N|     Randall Airport| 41.43191|-74.39156| 523| -5|  A|
|09J|Jekyll Island Air...|31.074472|-81.42778|  11| -4|  A|
+---+--------------------+---------+---------+----+---+---+
only showing top 5 rows



In [132]:
df_airports.printSchema()

df_planes.printSchema()

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: float (nullable = true)
 |-- lon: float (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)

root
 |-- tailnum: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: integer (nullable = true)
 |-- engine: string (nullable = true)



# Pergunta 1

In [133]:
def _view ():
    return df_airports_final.createOrReplaceTempView('airports_final')

def view_view(_col):
    return spark.getOrCreate().sql(f"select {_col}, Count(*) from airports_final Group By {_col}").show()

def view_all(_col):
    _view()
    return view_view(_col)


def _view_planes ():
    return df_planes_final.createOrReplaceTempView('planes_final')

def view_view_planes(_col):
    return spark.getOrCreate().sql(f"select {_col}, Count(*) from planes_final Group By {_col}").show()

def view_all_planes(_col):
    _view_planes()
    return view_view_planes(_col)

In [134]:
df_airports_final = df_airports.withColumn("alt", (
                                    F.when((F.col('alt') < 0), 0)
                                            .otherwise(F.col('alt'))))
                                    

In [135]:
df_airports_final.groupBy("alt").count().distinct().orderBy("alt", ascending=True).show()

+---+-----+
|alt|count|
+---+-----+
|  0|   51|
|  1|    2|
|  2|    1|
|  3|    2|
|  4|    3|
|  5|    3|
|  6|    3|
|  7|    7|
|  8|    8|
|  9|    7|
| 10|   11|
| 11|    6|
| 12|    8|
| 13|   12|
| 14|   11|
| 15|   11|
| 16|    3|
| 17|    7|
| 18|   10|
| 19|    6|
+---+-----+
only showing top 20 rows



# Pergunta 2

In [136]:
df_airports_final.groupBy("dst").count().distinct().orderBy("dst", ascending=True).show()

+---+-----+
|dst|count|
+---+-----+
|  A| 1329|
|  N|   23|
|  U|   45|
+---+-----+



In [137]:
df_airports_final = df_airports_final.withColumn("dst", (
                                                F.when((F.col("tz").between(-7,-5)), "A")
                                                    .otherwise(F.col("dst"))))
df_airports_final.createOrReplaceTempView('airports_final')

In [138]:
print("Sem o Codigo da Pergunta 2")
spark.getOrCreate().sql("select dst, Count(*) from airports Group By dst").show()

print("Com o Codigo da Pergunta 2")
spark.getOrCreate().sql("select dst, Count(*) from airports_final Group By dst").show()

Sem o Codigo da Pergunta 2
+---+--------+
|dst|count(1)|
+---+--------+
|  U|      45|
|  A|    1329|
|  N|      23|
+---+--------+

Com o Codigo da Pergunta 2
+---+--------+
|dst|count(1)|
+---+--------+
|  U|       8|
|  A|    1380|
|  N|       9|
+---+--------+



# Pergunta 3

In [139]:
df_airports_final = df_airports_final.withColumn("dst", (
                                                F.when((F.col("dst") == "U"), "A")
                                                .otherwise(F.col('dst'))))

df_airports_final.createOrReplaceTempView('airports_final')

In [140]:
spark.getOrCreate().sql("select dst, Count(*) from airports_final Group By dst").show()

+---+--------+
|dst|count(1)|
+---+--------+
|  A|    1388|
|  N|       9|
+---+--------+



# Pergunta 4

In [141]:
df_airports_final = df_airports_final.withColumn("region", (
                                       F.when((F.col("lon") < -124), "ALASKA")
                                        .when((F.col("lon") > -50) |
                                              (F.col("lat") < 24), "OFFSHORE")
                                        .when((F.col("lon") <= -95) & 
                                              (F.col("lon").between(-124, -50)), "MAINLAND-WEST")
                                        .when((F.col("lon") > -95) & 
                                              (F.col("lon").between(-124, -50)), "MAINLAND-EAST" )
                                        .otherwise("NaN")))

df_airports_final.createOrReplaceTempView('airports_final')

In [142]:
spark.getOrCreate().sql("select region, Count(*) from airports_final Group By region").show()

+-------------+--------+
|       region|count(1)|
+-------------+--------+
|       ALASKA|     261|
|     OFFSHORE|       4|
|MAINLAND-EAST|     696|
|MAINLAND-WEST|     436|
+-------------+--------+



# Pergunta 5

In [143]:
AP = ["Airport", "Tradeport", "Heliport", "Airpor","Arpt"]
AD = ["Aerodrome"]
AK = ["Airpark", "Aero Park"]
AS = ["Station" ,"Air Station"]
FL = ["Field", "Fld"]

df_airports_final = df_airports_final.withColumn("type", (
                                                F.when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", AP)))), "AP")
                                                 .when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", AD)))), "AD")
                                                 .when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", AK)))), "AK")
                                                 .when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", AS)))), "AS")
                                                 .when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", FL)))), "FL")
                                                 .otherwise("NaN")
                                                 ))

view_all("type")

+----+--------+
|type|count(1)|
+----+--------+
|  AD|       1|
| NaN|     663|
|  AS|      19|
|  FL|      78|
|  AK|      12|
|  AP|     624|
+----+--------+



# Pergunta 6

In [144]:
pergunta6 = "military"

lista = ["Base", "Aaf", "Afs", "Ahp", "Afb", "LRRS", "Lrrs", "Arb", "Naf", "NAS", "Nas", "Jrb", "Ns",
"As", "Cgas", "Angb"]

df_airports_final = df_airports_final.withColumn(pergunta6, (
                                                F.when((F.col("name").rlike(r'|'.join(map(lambda x : f'^{x} | {x} | {x}$', lista)))), True)
                                                .otherwise(False)))

view_all(pergunta6)

+--------+--------+
|military|count(1)|
+--------+--------+
|    true|     161|
|   false|    1236|
+--------+--------+



# Pergunta 7

In [145]:
I = ["International", "Intl", "Intercontinental"]
N = ["National", "Natl"]
R = ["Regional", "Reigonal", "Rgnl", "County", "Metro", "Metropolitan"]
M = ["Municipal", "Muni", "City"]

df_airports_final = df_airports_final.withColumn("administration", (
                                                F.when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", I)))), "I")
                                                 .when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", N)))), "N")
                                                 .when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", R)))), "R")
                                                 .when((F.col("name").rlike(r'|'.join(map(lambda x : f".*({x}).*", M)))), "M")
                                                 .otherwise("NaN")
                                                 ))

view_all("administration")

+--------------+--------+
|administration|count(1)|
+--------------+--------+
|             M|     180|
|             N|       5|
|             R|     287|
|           NaN|     761|
|             I|     164|
+--------------+--------+



# <center> Planes

# Pergunta 1

In [146]:
df_planes_final = df_planes_final.withColumn("tailchar",
    F.regexp_replace(F.col("tailnum"), '[0-9]|^N', ""))





In [147]:
df_planes_final.show()

+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+----+-----------+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar| age|engine_type|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+----+-----------+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW|24.0|        FAN|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      US|23.0|        FAN|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW|23.0|        FAN|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW|23.0|        FAN|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      US|23.0|        FAN|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW|23.0|        FAN|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-f

In [148]:
df_planes_final.groupBy("tailchar").count().distinct().orderBy("tailchar", ascending=True).show()

+--------+-----+
|tailchar|count|
+--------+-----+
|        |  298|
|       A|   46|
|      AA|   67|
|      AG|   10|
|      AS|  138|
|      AT|    4|
|      AW|   87|
|      AY|    4|
|       B|   28|
|      BR|    6|
|       C|   20|
|      CA|   21|
|      CB|    2|
|      CT|    1|
|       D|   16|
|      DA|   46|
|      DE|    2|
|      DH|    2|
|      DL|   61|
|      DN|  118|
+--------+-----+
only showing top 20 rows



# Pergunta 2

In [110]:
df_planes_final = df_planes_final.withColumn("year", (
                                    F.when((F.col("year") == 0), 1996)
                                     .otherwise(F.col("year"))))

df_planes.select('*').show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null

In [28]:
df_planes.groupBy("year").count().distinct().orderBy("year", ascending=True).show()

+----+-----+
|year|count|
+----+-----+
|null|   60|
|   0|    1|
|1959|    1|
|1963|    1|
|1968|    1|
|1975|    2|
|1976|    1|
|1980|    1|
|1984|    5|
|1985|   15|
|1986|   13|
|1987|   23|
|1988|   29|
|1989|   21|
|1990|   42|
|1991|   48|
|1992|   78|
|1993|   41|
|1994|   39|
|1995|   54|
+----+-----+
only showing top 20 rows



# Pergunta 3

In [112]:
# Criando tabela ordenada por Manufacturer e Model
df_ordered = df_planes.groupBy("manufacturer",'model').min('year').orderBy('manufacturer','model')



# Renomeando colunas
df_ordered = df_ordered.withColumnRenamed('manufacturer', 'manufacturer1').withColumnRenamed('model', 'model1') \
.withColumnRenamed('min(year)', 'year1')



# Criando tabela ordenada somente por manufacturer
df_ordered2 = df_planes.groupBy('manufacturer').min('year').orderBy('manufacturer')



# Renomeando colunas
df_ordered2 = df_ordered2.withColumnRenamed('manufacturer', 'manufacturer2') \
.withColumnRenamed('min(year)', 'year2')



# Fazendo left join da tabela df_planes com a tabela df_ordered



df_final = df_planes.join(df_ordered, (df_planes.manufacturer == df_ordered.manufacturer1) &
(df_planes.model == df_ordered.model1),'left')



# Fazendo left join da tabela df_final com a df_ordered2



df_final = df_final.join(df_ordered2, df_final.manufacturer == df_ordered2.manufacturer2 , 'left')



# Modificando a coluna year para inputar os anos



df_final = df_final.withColumn('year',
F.when((F.col('year').isNull()) & (F.col('year1').isNull()) & (F.col('year2').isNull()), "NaN")
.when((F.col('year').isNull()) & (F.col('year1').isNull()), F.col('year2'))
.when(F.col('year').isNull(), F.col('year1'))
.otherwise(F.col('year'))
)
# Dropando as colunas criadas pelo join



df_final = df_final.drop('manufacturer1','manufacturer2','model1','year1','year2')



df_final.show()



df_final.select("year","manufacturer","model").filter(F.col('year') == "NaN").show()



df_planes_final = df_final

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null

# pergunta 4

In [115]:
df_planes_final = df_planes_final.withColumn('age', 2022 - F.col('year'))

df_planes_final.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+----+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|tailchar| age|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+----+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|24.0|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US|23.0|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|23.0|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|23.0|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US|23.0|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|23.0|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDU

In [116]:
df_planes_final.groupBy("age").count().distinct().orderBy("age", ascending=False).show()

+------+-----+
|   age|count|
+------+-----+
|   NaN|    2|
|2022.0|    1|
|  63.0|    1|
|  59.0|    1|
|  54.0|    1|
|  47.0|    2|
|  46.0|    1|
|  42.0|    1|
|  38.0|    6|
|  37.0|   15|
|  36.0|   13|
|  35.0|   23|
|  34.0|   29|
|  33.0|   21|
|  32.0|   42|
|  31.0|   48|
|  30.0|   78|
|  29.0|   42|
|  28.0|   45|
|  27.0|   55|
+------+-----+
only showing top 20 rows



# Pergunta 5

In [117]:
df_planes_final = df_planes_final.withColumn('type',(
                            F.when((F.col('type') ==  'Fixed wing multi engine'), "MULTI_ENG")
                             .when((F.col('type') ==  'Fixed wing single engine'), "SINGLE_ENG")
                             .when((F.col('type') ==  'Rotorcraft'), "ROTORCRAFT")
                            .otherwise(F.col('type'))
))

df_planes_final.groupBy('type').count().show()

+----------+-----+
|      type|count|
+----------+-----+
|SINGLE_ENG|   10|
| MULTI_ENG| 2615|
|ROTORCRAFT|    3|
+----------+-----+



# Pergunta 6

In [118]:
df_planes_final = df_planes_final.withColumn('manufacturer', (
                                                           F.when((F.col('manufacturer').contains('AIRBUS')), "AIRBUS")
                                                            .when((F.col('manufacturer').contains('BOEING')), "BOEING")
                                                            .when((F.col('manufacturer').contains('BOMBARDIER')), "BOMBARDIER")
                                                            .when((F.col('manufacturer').contains('EMBRAER')), "EMBRAER")
                                                            .when((F.col('manufacturer').contains('SIKORSKY')), "SIKORSKY")
                                                            .when((F.col('manufacturer').contains('CANADAIR')), "CANADAIR")
                                                            .when((F.col('manufacturer').contains('PIPER')), "PIPER")
                                                            .when((F.col('manufacturer').contains('MCDONNELL DOUGLAS')), "MCDONNELL DOUGLAS")
                                                            .when((F.col('manufacturer').contains('CIRRUS')), "CIRRUS")
                                                            .when((F.col('manufacturer').contains('BELL')), "BELL")
                                                            .when((F.col('manufacturer').contains('KILDALL GARY')), "KILDALL GARY")
                                                            .when((F.col('manufacturer').contains('LAMBERT RICHARD')), "LAMBERT RICHARD")
                                                            .when((F.col('manufacturer').contains('BARKER JACK')), "BARKER JACK")
                                                            .when((F.col('manufacturer').contains('ROBINSON HELICOPTER')), "ROBINSON HELICOPTER")
                                                            .when((F.col('manufacturer').contains('GULFSTREAM')), "GULFSTREAM")
                                                            .when((F.col('manufacturer').contains('MARZ BARRY')), "MARZ BARRY")
                                                            .otherwise(F.col('manufacturer'))
))

In [119]:
df_planes_final.groupBy('manufacturer').count().orderBy('count',ascending=False).show()

+-------------------+-----+
|       manufacturer|count|
+-------------------+-----+
|             BOEING| 1460|
|             AIRBUS|  798|
|         BOMBARDIER|  214|
|  MCDONNELL DOUGLAS|   96|
|            EMBRAER|   37|
|           CANADAIR|    8|
|             CESSNA|    4|
|              PIPER|    2|
|        BARKER JACK|    1|
|         GULFSTREAM|    1|
|           SIKORSKY|    1|
|       KILDALL GARY|    1|
|             CIRRUS|    1|
|         MARZ BARRY|    1|
|    LAMBERT RICHARD|    1|
|ROBINSON HELICOPTER|    1|
|               BELL|    1|
+-------------------+-----+



# Pergunta 7

In [35]:
df_planes_final.groupBy('model').count().show(9999)

+--------------+-----+
|         model|count|
+--------------+-----+
|       737-76Q|    3|
|     PA-28-180|    1|
|       737-3A4|    1|
|       767-3P6|    4|
|     EMB-120ER|   34|
|       757-2S7|    4|
|      A321-231|   66|
|       737-705|    2|
|          421C|    1|
|       737-924|   12|
|      A320-211|   27|
|       767-3CB|    3|
|      A320-232|  255|
|       757-28A|    1|
|      A319-112|   56|
|       737-73V|    1|
|       EMB-120|    3|
|       737-824|  122|
|     FALCON-XP|    1|
|      A330-223|   11|
|   CL-600-2D24|   32|
|       757-223|   11|
|       757-232|   82|
|          172M|    1|
|       757-324|    4|
|         MD-88|    2|
|      A320-214|   86|
|       737-301|    2|
|     737-932ER|   30|
|       767-33A|    4|
|       757-231|    8|
|     737-990ER|   23|
|      A319-115|    1|
|       737-3K2|    2|
|       737-7H4|  360|
|       757-224|    1|
|      A320-231|    8|
|       767-322|    6|
|          SR22|    1|
|      A319-114|   53|
|       757

In [120]:
df_planes_final = (df_planes_final.withColumn("model",
F.regexp_replace(F.col("model"), '\s*\([^()]*\)\s*', "")
))

In [121]:
df_planes_final.select('*').show(9999)

+-------+----+----------+-------------------+-------------+-------+-----+-----+-------------+--------+------+
|tailnum|year|      type|       manufacturer|        model|engines|seats|speed|       engine|tailchar|   age|
+-------+----+----------+-------------------+-------------+-------+-----+-----+-------------+--------+------+
| N102UW|1998| MULTI_ENG|             AIRBUS|     A320-214|      2|  182| null|    Turbo-fan|      UW|  24.0|
| N103US|1999| MULTI_ENG|             AIRBUS|     A320-214|      2|  182| null|    Turbo-fan|      US|  23.0|
| N104UW|1999| MULTI_ENG|             AIRBUS|     A320-214|      2|  182| null|    Turbo-fan|      UW|  23.0|
| N105UW|1999| MULTI_ENG|             AIRBUS|     A320-214|      2|  182| null|    Turbo-fan|      UW|  23.0|
| N107US|1999| MULTI_ENG|             AIRBUS|     A320-214|      2|  182| null|    Turbo-fan|      US|  23.0|
| N108UW|1999| MULTI_ENG|             AIRBUS|     A320-214|      2|  182| null|    Turbo-fan|      UW|  23.0|
| N109UW|1

# Pergunta 8

In [122]:
df_planes_final.groupBy('seats').count().orderBy('seats', ascending=True).show(100)

+-----+-----+
|seats|count|
+-----+-----+
|    2|    5|
|    4|    3|
|    5|    2|
|    6|    1|
|    7|    1|
|    8|    1|
|   14|    1|
|   20|    1|
|   32|   37|
|   55|   92|
|   80|   98|
|   95|   32|
|  100|   21|
|  128|    1|
|  140|  441|
|  142|   67|
|  145|   53|
|  147|   24|
|  149|  444|
|  151|   17|
|  172|   44|
|  178|  184|
|  179|  129|
|  182|  163|
|  189|   73|
|  191|  114|
|  199|   43|
|  200|  255|
|  222|   53|
|  249|    1|
|  275|   24|
|  290|    4|
|  300|   12|
|  330|   73|
|  377|   19|
|  379|   91|
|  400|    2|
|  442|    1|
|  450|    1|
+-----+-----+



In [123]:
df_planes_final = df_planes_final.withColumn('speed',
                                             F.when((F.col('speed').isNull()) &
                                                   (F.col('seats').isNotNull()), F.ceil(F.expr('seats/0.36')))
                                             .when(~F.col('speed').between(50, 150), 0)
                                             .otherwise(F.col('speed')))

In [40]:
df_planes_final.groupBy('speed').count().orderBy('speed', ascending=True).show(999)

+-----+-----+
|speed|count|
+-----+-----+
|    6|    4|
|   12|    1|
|   14|    1|
|   17|    1|
|   39|    1|
|   56|    1|
|   89|   37|
|   90|    2|
|  107|    1|
|  108|    1|
|  112|    1|
|  126|    1|
|  153|   92|
|  223|   98|
|  264|   32|
|  278|   21|
|  356|    1|
|  389|  441|
|  395|   67|
|  403|   53|
|  409|   24|
|  414|  444|
|  420|   17|
|  478|   44|
|  495|  184|
|  498|  129|
|  506|  163|
|  525|   73|
|  531|  114|
|  553|   43|
|  556|  255|
|  617|   53|
|  692|    1|
|  764|   24|
|  806|    4|
|  834|   12|
|  917|   73|
| 1048|   19|
| 1053|   91|
| 1112|    2|
| 1228|    1|
| 1250|    1|
+-----+-----+



In [41]:
df_planes_final.printSchema()

root
 |-- tailnum: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: decimal(13,0) (nullable = true)
 |-- engine: string (nullable = true)
 |-- age: integer (nullable = true)



# pergunta 9

In [42]:
df_planes_final.groupBy('engine').count().show()

+-------------+-----+
|       engine|count|
+-------------+-----+
|    Turbo-jet|  450|
|      4 Cycle|    1|
|    Turbo-fan| 2127|
|   Turbo-prop|   37|
|Reciprocating|   10|
|  Turbo-shaft|    3|
+-------------+-----+



In [124]:
df_planes_final = df_planes_final.withColumn('engine_type', (
                                                            F.when((F.col('engine').contains('Turbo-fan')), "FAN")
                                                            .when((F.col('engine').contains('Turbo-jet')), "JET")
                                                            .when((F.col('engine').contains('Turbo-prop')), "PROP")
                                                            .when((F.col('engine').contains('Turbo-shaft')), "SHAFT")
                                                            .when((F.col('engine').contains('4 Cycle')), "CYCLE")
                                                            .otherwise(None)))

In [44]:
df_planes_final.groupBy('engine_type').count().show()

+-----------+-----+
|engine_type|count|
+-----------+-----+
|       PROP|   37|
|       null|   10|
|      CYCLE|    1|
|        FAN| 2127|
|        JET|  450|
|      SHAFT|    3|
+-----------+-----+



In [45]:
df_planes_final.show()
df_planes.show()

+-------+----+---------+------------+--------+-------+-----+-----+---------+---+-----------+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|age|engine_type|
+-------+----+---------+------------+--------+-------+-----+-----+---------+---+-----------+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan| 24|        FAN|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan| 23|        FAN|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan| 23|        FAN|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan| 23|        FAN|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan| 23|        FAN|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan| 23|        FAN|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan| 23|        FAN|
| N110UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turb

# <center> Flights Dataset

In [46]:
df_flights.groupBy('hour').count().filter('hour == 0').show()
df_flights.groupBy('hour').count().filter('hour is NULL').show()
df_flights.groupBy('hour').count().orderBy('hour', ascending=True).show()

+----+-----+
|hour|count|
+----+-----+
|   0|   89|
+----+-----+

+----+-----+
|hour|count|
+----+-----+
|null|   48|
+----+-----+

+----+-----+
|hour|count|
+----+-----+
|null|   48|
|   0|   89|
|   1|   17|
|   2|    4|
|   5|  431|
|   6|  899|
|   7|  709|
|   8|  659|
|   9|  456|
|  10|  803|
|  11|  723|
|  12|  539|
|  13|  653|
|  14|  540|
|  15|  468|
|  16|  388|
|  17|  394|
|  18|  570|
|  19|  409|
|  20|  354|
+----+-----+
only showing top 20 rows



In [47]:
''' o dado (0) tem 89 registros
    o dado (null) tem 48 registro
    o esperado pos pergunta é 137
''' 


' o dado (0) tem 89 registros\n    o dado (null) tem 48 registro\n    o esperado pos pergunta é 137\n'

In [48]:
df_flights = df_flights.withColumn('hour', (
F.when((F.col('hour').isNull()), 0)
.otherwise(F.col('hour'))))

In [49]:
df_flights.groupBy('hour').count().filter('hour == 0').show()

+----+-----+
|hour|count|
+----+-----+
|   0|  137|
+----+-----+



In [50]:
df_flights.groupBy('minute').count().filter('minute == 0 or minute is NULL ' ).show()

+------+-----+
|minute|count|
+------+-----+
|  null|   48|
|     0|  178|
+------+-----+



In [51]:
df_flights = df_flights.withColumn('minute', (
F.when((F.col('minute').isNull()), 0)
.otherwise(F.col('minute'))))

In [52]:
df_flights.groupBy('minute').count().filter('minute == 0 or minute is NULL').show()

+------+-----+
|minute|count|
+------+-----+
|     0|  226|
+------+-----+



In [53]:
df_flights.groupBy('hour').count().filter('hour == 0').show()
df_flights.groupBy('minute').count().filter('minute == 0').show()


+----+-----+
|hour|count|
+----+-----+
|   0|  137|
+----+-----+

+------+-----+
|minute|count|
+------+-----+
|     0|  226|
+------+-----+



# Pergunta 2

In [54]:
df_flights.groupBy('hour').count().filter('hour == 0').show()
df_flights.groupBy('hour').count().filter('hour == 24').show()

+----+-----+
|hour|count|
+----+-----+
|   0|  137|
+----+-----+

+----+-----+
|hour|count|
+----+-----+
|  24|    1|
+----+-----+



In [55]:
df_flights = df_flights.withColumn('hour', (
F.when((F.col('hour') == 24), 0)
.otherwise(F.col('hour'))))

In [56]:
df_flights.groupBy('hour').count().filter('hour == 0').show()

+----+-----+
|hour|count|
+----+-----+
|   0|  138|
+----+-----+



# Pergunta 3

In [57]:

df_flights = df_flights.withColumn('dep_datetime', 
F.concat_ws(':',
            F.concat_ws(' ',
                        F.concat_ws('-',F.col('year'), F.col('month'), F.col('day')),
            F.col('hour')), 
F.col('minute')).cast('timestamp'))

In [58]:
df_flights.select('dep_datetime').show()
df_flights.select('*').show()

+-------------------+
|       dep_datetime|
+-------------------+
|2014-12-08 06:58:00|
|2014-01-22 10:40:00|
|2014-03-09 14:43:00|
|2014-04-09 17:05:00|
|2014-03-09 07:54:00|
|2014-01-15 10:37:00|
|2014-07-02 08:47:00|
|2014-05-12 16:55:00|
|2014-04-19 12:36:00|
|2014-11-19 18:12:00|
|2014-11-08 16:53:00|
|2014-08-03 11:20:00|
|2014-10-30 08:11:00|
|2014-11-12 23:46:00|
|2014-10-31 13:14:00|
|2014-01-29 20:09:00|
|2014-12-17 20:15:00|
|2014-08-11 10:17:00|
|2014-01-13 21:56:00|
|2014-06-05 17:33:00|
+-------------------+
only showing top 20 rows

+----+-----+---+----+------+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|year|month|day|hour|minute|dep_time|arr_time|dep_delay|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|
+----+-----+---+----+------+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|2014|   12|  8|   6|    5

# Pergunta 4

In [59]:
df_flights.groupBy('dep_time').count().orderBy('dep_time', ascending= True).show()
df_flights.groupBy('dep_time').count().orderBy('dep_time', ascending= False).show()

+--------+-----+
|dep_time|count|
+--------+-----+
|       1|    6|
|    1000|    8|
|    1001|   16|
|    1002|    5|
|    1003|   12|
|    1004|    9|
|    1005|    6|
|    1006|   17|
|    1007|   10|
|    1008|    9|
|    1009|   18|
|     101|    1|
|    1010|   12|
|    1011|   10|
|    1012|   18|
|    1013|   12|
|    1014|   13|
|    1015|   17|
|    1016|   15|
|    1017|   17|
+--------+-----+
only showing top 20 rows

+--------+-----+
|dep_time|count|
+--------+-----+
|      NA|   48|
|     959|    9|
|     958|   12|
|     957|   14|
|     956|    7|
|     955|   10|
|     954|   12|
|     953|    3|
|     952|    7|
|     951|    6|
|     950|   13|
|     949|   15|
|     948|   13|
|     947|   12|
|     946|   15|
|     945|   14|
|     944|   12|
|     943|   10|
|     942|    8|
|     941|    7|
+--------+-----+
only showing top 20 rows



In [60]:
df_flights = (df_flights.withColumn('dep_time',
F.when(F.col('dep_time') == 'NA',
F.concat(F.col('hour'),
F.lpad(F.col('minute'), 2, '0')))
.otherwise(F.col('dep_time'))
))

In [61]:
df_flights.groupBy('dep_time').count().orderBy('dep_time', ascending= True).show()
df_flights.groupBy('dep_time').count().orderBy('dep_time', ascending= False).show()

+--------+-----+
|dep_time|count|
+--------+-----+
|     000|   48|
|       1|    6|
|    1000|    8|
|    1001|   16|
|    1002|    5|
|    1003|   12|
|    1004|    9|
|    1005|    6|
|    1006|   17|
|    1007|   10|
|    1008|    9|
|    1009|   18|
|     101|    1|
|    1010|   12|
|    1011|   10|
|    1012|   18|
|    1013|   12|
|    1014|   13|
|    1015|   17|
|    1016|   15|
+--------+-----+
only showing top 20 rows

+--------+-----+
|dep_time|count|
+--------+-----+
|     959|    9|
|     958|   12|
|     957|   14|
|     956|    7|
|     955|   10|
|     954|   12|
|     953|    3|
|     952|    7|
|     951|    6|
|     950|   13|
|     949|   15|
|     948|   13|
|     947|   12|
|     946|   15|
|     945|   14|
|     944|   12|
|     943|   10|
|     942|    8|
|     941|    7|
|     940|    8|
+--------+-----+
only showing top 20 rows



# Pergunta 5

In [62]:
df_flights.groupBy('dep_delay').count().filter('dep_delay == 0').show()
df_flights.groupBy('dep_delay').count().filter('dep_delay is NULL').show()
df_flights.groupBy('dep_delay').count().orderBy('dep_delay', ascending= True).show()

+---------+-----+
|dep_delay|count|
+---------+-----+
|        0|  598|
+---------+-----+

+---------+-----+
|dep_delay|count|
+---------+-----+
|     null|   48|
+---------+-----+

+---------+-----+
|dep_delay|count|
+---------+-----+
|     null|   48|
|      -19|    1|
|      -18|    3|
|      -17|    4|
|      -16|    5|
|      -15|   10|
|      -14|   14|
|      -13|   37|
|      -12|   63|
|      -11|   80|
|      -10|  196|
|       -9|  208|
|       -8|  319|
|       -7|  471|
|       -6|  625|
|       -5|  742|
|       -4|  838|
|       -3|  852|
|       -2|  760|
|       -1|  666|
+---------+-----+
only showing top 20 rows



In [63]:
df_flights = df_flights.withColumn('dep_delay', (
F.when((F.col('dep_delay').isNull()), 0)
                                  .otherwise(F.col('dep_delay'))))

In [64]:
df_flights.groupBy('dep_delay').count().filter('dep_delay == 0').show()
df_flights.groupBy('dep_delay').count().filter('dep_delay is NULL').show()
df_flights.groupBy('dep_delay').count().orderBy('dep_delay', ascending= True).show()

+---------+-----+
|dep_delay|count|
+---------+-----+
|        0|  646|
+---------+-----+

+---------+-----+
|dep_delay|count|
+---------+-----+
+---------+-----+

+---------+-----+
|dep_delay|count|
+---------+-----+
|      -19|    1|
|      -18|    3|
|      -17|    4|
|      -16|    5|
|      -15|   10|
|      -14|   14|
|      -13|   37|
|      -12|   63|
|      -11|   80|
|      -10|  196|
|       -9|  208|
|       -8|  319|
|       -7|  471|
|       -6|  625|
|       -5|  742|
|       -4|  838|
|       -3|  852|
|       -2|  760|
|       -1|  666|
|        0|  646|
+---------+-----+
only showing top 20 rows



# pergunta 6

In [65]:
df_flights.groupBy('arr_delay').count().filter('arr_delay == 0').show()
df_flights.groupBy('arr_delay').count().filter('arr_delay is NULL').show()
df_flights.groupBy('arr_delay').count().orderBy('arr_delay', ascending= True).show()
df_flights.groupBy('arr_delay').count().orderBy('arr_delay', ascending= False).show()

+---------+-----+
|arr_delay|count|
+---------+-----+
|        0|  252|
+---------+-----+

+---------+-----+
|arr_delay|count|
+---------+-----+
|     null|   75|
+---------+-----+

+---------+-----+
|arr_delay|count|
+---------+-----+
|     null|   75|
|      -58|    1|
|      -53|    1|
|      -50|    1|
|      -48|    1|
|      -47|    2|
|      -46|    1|
|      -45|    2|
|      -44|    2|
|      -43|    2|
|      -42|    6|
|      -41|    4|
|      -40|    7|
|      -39|    3|
|      -38|   14|
|      -37|    7|
|      -36|   15|
|      -35|   14|
|      -34|   17|
|      -33|   14|
+---------+-----+
only showing top 20 rows

+---------+-----+
|arr_delay|count|
+---------+-----+
|      900|    1|
|      720|    1|
|      643|    1|
|      375|    1|
|      371|    1|
|      358|    1|
|      322|    1|
|      318|    1|
|      314|    1|
|      284|    1|
|      279|    1|
|      278|    1|
|      268|    1|
|      261|    1|
|      248|    1|
|      242|    1|
|      237|    1|


# Pergunta 7

In [66]:
df_flights = df_flights.withColumn('arr_delay', (
F.when((F.col('arr_delay').isNull()), 0)
                                  .otherwise(F.col('arr_delay'))))

In [67]:
df_flights.groupBy('arr_delay').count().filter('arr_delay == 0').show()
df_flights.groupBy('arr_delay').count().filter('arr_delay is NULL').show()
df_flights.groupBy('arr_delay').count().orderBy('arr_delay', ascending= True).show()
df_flights.groupBy('arr_delay').count().orderBy('arr_delay', ascending= False).show()

+---------+-----+
|arr_delay|count|
+---------+-----+
|        0|  327|
+---------+-----+

+---------+-----+
|arr_delay|count|
+---------+-----+
+---------+-----+

+---------+-----+
|arr_delay|count|
+---------+-----+
|      -58|    1|
|      -53|    1|
|      -50|    1|
|      -48|    1|
|      -47|    2|
|      -46|    1|
|      -45|    2|
|      -44|    2|
|      -43|    2|
|      -42|    6|
|      -41|    4|
|      -40|    7|
|      -39|    3|
|      -38|   14|
|      -37|    7|
|      -36|   15|
|      -35|   14|
|      -34|   17|
|      -33|   14|
|      -32|   17|
+---------+-----+
only showing top 20 rows

+---------+-----+
|arr_delay|count|
+---------+-----+
|      900|    1|
|      720|    1|
|      643|    1|
|      375|    1|
|      371|    1|
|      358|    1|
|      322|    1|
|      318|    1|
|      314|    1|
|      284|    1|
|      279|    1|
|      278|    1|
|      268|    1|
|      261|    1|
|      248|    1|
|      242|    1|
|      237|    1|
|      236|    1|


In [68]:
df_flights = (df_flights.withColumn('arr_delay',
F.when(F.col('arr_delay').isNull(), 0)
.otherwise(F.col('arr_delay'))

))



In [69]:
df_flights = df_flights.drop('year','month','model1','day','hour','minute')

In [70]:
df_flights.select('*').show()

+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|dep_time|arr_time|dep_delay|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|
+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|     658|     935|       -7|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|
|    1040|    1505|        5|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|
|    1443|    1652|       -2|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|
|    1705|    1839|       45|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|2014-04-09 17:05:00|
|     754|    1015|       -1|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|2014-03-09 07:54:00|
|    1037|    1352|        7|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|2014

# Pergunta 8

In [71]:
df_flights.groupBy('distance').count().orderBy('distance', ascending= True).show()
df_flights.groupBy('distance').count().orderBy('distance', ascending= False).show()

+--------+-----+
|distance|count|
+--------+-----+
|      93|    5|
|     106|   41|
|     116|   54|
|     129|  301|
|     224|  105|
|     241|   10|
|     344|    7|
|     444|   13|
|     479|   93|
|     543|  121|
|     550|  305|
|     564|    8|
|     569|  156|
|     605|  190|
|     630|  171|
|     671|  193|
|     672|   20|
|     679|  482|
|     680|   77|
|     689|  225|
+--------+-----+
only showing top 20 rows

+--------+-----+
|distance|count|
+--------+-----+
|    2724|   19|
|    2717|   29|
|    2701|   30|
|    2688|   40|
|    2677|   86|
|    2640|   63|
|    2631|    7|
|    2607|   12|
|    2603|   57|
|    2562|   28|
|    2554|   31|
|    2537|   33|
|    2520|   11|
|    2496|   96|
|    2454|   46|
|    2434|   21|
|    2422|  127|
|    2406|    9|
|    2402|  122|
|    2378|   72|
+--------+-----+
only showing top 20 rows



In [72]:
df_flights = df_flights.withColumn('air_time_projected', ((F.col('distance') * 0.1) + 20).cast(IntegerType()))


In [73]:
df_flights.groupBy('air_time_projected').count().orderBy('air_time_projected', ascending= True).show(3)
df_flights.groupBy('air_time_projected').count().orderBy('air_time_projected', ascending= False).show(3)

+------------------+-----+
|air_time_projected|count|
+------------------+-----+
|                29|    5|
|                30|   41|
|                31|   54|
+------------------+-----+
only showing top 3 rows

+------------------+-----+
|air_time_projected|count|
+------------------+-----+
|               292|   19|
|               291|   29|
|               290|   30|
+------------------+-----+
only showing top 3 rows



# Pergunta 9

In [74]:
df_flights.groupBy('origin','dest').count().orderBy('count', ascending= False).show()
df_flights.select('*').show(1)

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   SEA| SFO|  482|
|   SEA| LAX|  450|
|   SEA| ANC|  380|
|   SEA| LAS|  364|
|   SEA| DEN|  351|
|   SEA| PHX|  321|
|   PDX| SFO|  305|
|   SEA| ORD|  282|
|   SEA| DFW|  245|
|   PDX| DEN|  235|
|   SEA| SLC|  225|
|   SEA| SJC|  213|
|   SEA| OAK|  213|
|   PDX| PHX|  209|
|   SEA| SMF|  190|
|   SEA| SAN|  180|
|   PDX| SLC|  171|
|   SEA| IAH|  169|
|   SEA| MSP|  166|
|   PDX| LAX|  165|
+------+----+-----+
only showing top 20 rows

+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+
|dep_time|arr_time|dep_delay|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|
+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+
|     658|     935|       -7|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-1

In [75]:
df_flights_ATE = df_flights.groupBy(F.col("origin"),F.col("dest")).agg(F.avg(F.col('air_time')).cast(IntegerType()).alias('air_time_expected'))

In [76]:
df_flights.show()
df_flights_ATE.show()

+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+
|dep_time|arr_time|dep_delay|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|
+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+
|     658|     935|       -7|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|               115|
|    1040|    1505|        5|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|               287|
|    1443|    1652|       -2|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|                87|
|    1705|    1839|       45|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|2014-04-09 17:05:00|                76|
|     754|    1015|       -1|        1|     AS| N612AS|   522|   SEA| BUR|  

In [77]:
df_flights.show()

+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+
|dep_time|arr_time|dep_delay|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|
+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+
|     658|     935|       -7|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|               115|
|    1040|    1505|        5|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|               287|
|    1443|    1652|       -2|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|                87|
|    1705|    1839|       45|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|2014-04-09 17:05:00|                76|
|     754|    1015|       -1|        1|     AS| N612AS|   522|   SEA| BUR|  

In [78]:
df_flights_ATE = (df_flights_ATE.withColumnRenamed('origin', 'O1')
.withColumnRenamed('dest', 'D1')
.withColumnRenamed('air_time_expected', 'air_time_expected')

)
print('modified columns:', df_flights_ATE.columns)



df_flights_ATE.show()

modified columns: ['O1', 'D1', 'air_time_expected']
+---+---+-----------------+
| O1| D1|air_time_expected|
+---+---+-----------------+
|SEA|RNO|               74|
|SEA|DTW|              219|
|SEA|CLE|              233|
|SEA|LAX|              126|
|PDX|SEA|               34|
|SEA|BLI|               22|
|PDX|IAH|              213|
|PDX|PHX|              130|
|SEA|SLC|               88|
|SEA|SBA|              118|
|SEA|BWI|              269|
|PDX|IAD|              267|
|PDX|SFO|               85|
|SEA|KOA|              347|
|PDX|MCI|              174|
|SEA|SJC|              103|
|SEA|ABQ|              142|
|SEA|SAT|              208|
|PDX|ONT|              111|
|SEA|LAS|              118|
+---+---+-----------------+
only showing top 20 rows



In [79]:
#df_final = df_planes.join(df_ordered, (df_planes.manufacturer == df_ordered.manufacturer1) &
#(df_planes.model == df_ordered.model1),'left')

df_flights = df_flights.join(df_flights_ATE, (df_flights.origin == df_flights_ATE.O1) &
    (df_flights.dest == df_flights_ATE.D1), "left")

In [80]:
df_flights.show()

+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+---+---+-----------------+
|dep_time|arr_time|dep_delay|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected| O1| D1|air_time_expected|
+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+---+---+-----------------+
|     658|     935|       -7|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|               115|SEA|LAX|              126|
|    1040|    1505|        5|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|               287|SEA|HNL|              343|
|    1443|    1652|       -2|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|                87|SEA|SFO|              101|
|    1705|    1839|       45|       34|     WN| N360

In [81]:
df_flights = df_flights.drop('O1','D1')

In [82]:
df_flights.show()

+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+
|dep_time|arr_time|dep_delay|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|air_time_projected|air_time_expected|
+--------+--------+---------+---------+-------+-------+------+------+----+--------+--------+-------------------+------------------+-----------------+
|     658|     935|       -7|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|               115|              126|
|    1040|    1505|        5|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|               287|              343|
|    1443|    1652|       -2|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|                87|              101|
|    1705|    1839|       45|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|2014-04-

In [83]:
del df_flights_ATE

# pergunta 10

In [84]:
df_flights.groupBy('air_time').count().orderBy('air_time', ascending= True).show(5)
df_flights.groupBy('air_time').count().orderBy('air_time', ascending= False).show(5)

+--------+-----+
|air_time|count|
+--------+-----+
|    null|   75|
|      20|    3|
|      24|    4|
|      25|   10|
|      26|   20|
+--------+-----+
only showing top 5 rows

+--------+-----+
|air_time|count|
+--------+-----+
|     409|    1|
|     403|    1|
|     399|    1|
|     396|    1|
|     392|    1|
+--------+-----+
only showing top 5 rows



In [85]:
df_flights= df_flights.withColumn('air_time',(
    F.when(F.col('air_time').isNull(), 
           F.greatest(F.col('air_time_projected'), F.col('air_time_expected')))
.otherwise(F.col('air_time'))))
    

In [86]:
df_flights.groupBy('air_time').count().orderBy('air_time', ascending= True).show(5)
df_flights.groupBy('air_time').count().orderBy('air_time', ascending= False).show(5)

+--------+-----+
|air_time|count|
+--------+-----+
|      20|    3|
|      24|    4|
|      25|   10|
|      26|   20|
|      27|   23|
+--------+-----+
only showing top 5 rows

+--------+-----+
|air_time|count|
+--------+-----+
|     409|    1|
|     403|    1|
|     399|    1|
|     396|    1|
|     392|    1|
+--------+-----+
only showing top 5 rows



In [87]:
df_flights.printSchema()

root
 |-- dep_time: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_delay: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- dep_datetime: timestamp (nullable = true)
 |-- air_time_projected: integer (nullable = true)
 |-- air_time_expected: integer (nullable = true)



# Pergunta 11

In [88]:
# Transforma a coluna dep_datetime em segundos com a função unix_timestamp e soma com a coluna air_time
# transformada em segundos, transformando novamente em timestamp depois
df_flights = df_flights.withColumn('sum',
(F.unix_timestamp("dep_datetime") +
(F.col('air_time') * 60).cast('int')).cast('timestamp'))



# Verifica aonde a coluna arr_time está nulo e troca o valor
df_flights = df_flights.withColumn('arr_time',
F.when(F.col('arr_time') == 'NA', F.format_string('%d%d', F.hour(F.col('sum')),
F.minute(F.col('sum'))))
.otherwise(F.col('arr_time')))



# Remove a coluna sum
df_flights = df_flights.drop('sum')



df_flights.toPandas()

,dep_time,arr_time,dep_delay,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected
0,658,935,-7,-5,VX,N846VA,1780,SEA,LAX,132,954,2014-12-08 06:58:00,115,126
1,1040,1505,5,5,AS,N559AS,851,SEA,HNL,360,2677,2014-01-22 10:40:00,287,343
2,1443,1652,-2,2,VX,N847VA,755,SEA,SFO,111,679,2014-03-09 14:43:00,87,101
3,1705,1839,45,34,WN,N360SW,344,PDX,SJC,83,569,2014-04-09 17:05:00,76,85
4,754,1015,-1,1,AS,N612AS,522,SEA,BUR,127,937,2014-03-09 07:54:00,113,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,2104,-4,-6,OO,N225AG,3458,SEA,SLC,89,689,2014-06-23 18:06:00,88,88
9996,2336,452,11,-13,AA,N3LEAA,1230,SEA,DFW,178,1660,2014-08-31 23:36:00,186,195
9997,904,1042,-1,-5,AS,N523AS,360,SEA,SMF,81,605,2014-08-08 09:04:00,80,82
9998,1441,1820,26,10,WN,N8647A,2857,SEA,ABQ,133,1180,2014-08-29 14:41:00,138,142


# Pergunta 12

In [89]:
df_flights = df_flights.withColumn('haul_duration', (
F.when((F.col('air_time').between(20,180)), "SHORT-HAUL")
 .when((F.col('air_time').between(181,360)), "MEDIUM-HAUL")
      .otherwise('LONG-HAUL')))

In [90]:
df_flights.groupBy('haul_duration').count().show()

+-------------+-----+
|haul_duration|count|
+-------------+-----+
|    LONG-HAUL|   54|
|  MEDIUM-HAUL| 3254|
|   SHORT-HAUL| 6692|
+-------------+-----+



# Pergunta 13

In [91]:
dep_datetime = 'dep_datetime'

df_flights.select(dep_datetime).show()

+-------------------+
|       dep_datetime|
+-------------------+
|2014-12-08 06:58:00|
|2014-01-22 10:40:00|
|2014-03-09 14:43:00|
|2014-04-09 17:05:00|
|2014-03-09 07:54:00|
|2014-01-15 10:37:00|
|2014-07-02 08:47:00|
|2014-05-12 16:55:00|
|2014-04-19 12:36:00|
|2014-11-19 18:12:00|
|2014-11-08 16:53:00|
|2014-08-03 11:20:00|
|2014-10-30 08:11:00|
|2014-11-12 23:46:00|
|2014-10-31 13:14:00|
|2014-01-29 20:09:00|
|2014-12-17 20:15:00|
|2014-08-11 10:17:00|
|2014-01-13 21:56:00|
|2014-06-05 17:33:00|
+-------------------+
only showing top 20 rows



In [92]:
dep_datetime = 'dep_datetime'


df_flights = df_flights.withColumn('dep_season', (
F.when((F.col(dep_datetime).between('2013-12-21 21:48:00','2014-03-20 15:32:59')), "WINTER")
 .when((F.col(dep_datetime).between('2014-03-20 15:33:00','2014-06-21 10:13:59')), "SPRING")
 .when((F.col(dep_datetime).between('2014-06-21 10:14:00','2014-09-23 02:03:59')), "SUMMER")
 .when((F.col(dep_datetime).between('2014-09-23 02:04:00','2014-12-21 21:47:59')), "FALL")
 .when((F.col(dep_datetime).between('2014-12-21 21:48:00','2015-03-20 15:32:59')), "WINTER")))

In [93]:
df_flights.select('dep_datetime','dep_season').show()
df_flights.groupBy('dep_season').count().show()
df_flights.select('dep_datetime','dep_season').filter('dep_season is NULL').show()

+-------------------+----------+
|       dep_datetime|dep_season|
+-------------------+----------+
|2014-12-08 06:58:00|      FALL|
|2014-01-22 10:40:00|    WINTER|
|2014-03-09 14:43:00|    WINTER|
|2014-04-09 17:05:00|    SPRING|
|2014-03-09 07:54:00|    WINTER|
|2014-01-15 10:37:00|    WINTER|
|2014-07-02 08:47:00|    SUMMER|
|2014-05-12 16:55:00|    SPRING|
|2014-04-19 12:36:00|    SPRING|
|2014-11-19 18:12:00|      FALL|
|2014-11-08 16:53:00|      FALL|
|2014-08-03 11:20:00|    SUMMER|
|2014-10-30 08:11:00|      FALL|
|2014-11-12 23:46:00|      FALL|
|2014-10-31 13:14:00|      FALL|
|2014-01-29 20:09:00|    WINTER|
|2014-12-17 20:15:00|      FALL|
|2014-08-11 10:17:00|    SUMMER|
|2014-01-13 21:56:00|    WINTER|
|2014-06-05 17:33:00|    SPRING|
+-------------------+----------+
only showing top 20 rows

+----------+-----+
|dep_season|count|
+----------+-----+
|    WINTER| 2149|
|    SPRING| 2560|
|      FALL| 2373|
|    SUMMER| 2918|
+----------+-----+

+------------+----------+
|de

# pergunta 14

In [94]:
df_flights.groupBy('dep_delay').count().orderBy('dep_delay', ascending= True).show(5)
df_flights.groupBy('dep_delay').count().orderBy('dep_delay', ascending= False).show(5)

+---------+-----+
|dep_delay|count|
+---------+-----+
|      -19|    1|
|      -18|    3|
|      -17|    4|
|      -16|    5|
|      -15|   10|
+---------+-----+
only showing top 5 rows

+---------+-----+
|dep_delay|count|
+---------+-----+
|      886|    1|
|      739|    1|
|      667|    1|
|      385|    1|
|      370|    1|
+---------+-----+
only showing top 5 rows



In [95]:
df_flights = df_flights.withColumn('dep_delay_category', (
F.when((F.col('dep_delay') < 0), "ANTECIPATED")
 .when((F.col('dep_delay') == 0), "INTIME")
 .when((F.col('dep_delay') >= 60 ), "MAJOR")
.otherwise("MINOR")))

In [96]:
df_flights.groupBy('dep_delay_category').count().show()

+------------------+-----+
|dep_delay_category|count|
+------------------+-----+
|       ANTECIPATED| 5894|
|             MAJOR|  395|
|             MINOR| 3065|
|            INTIME|  646|
+------------------+-----+



In [149]:
(df_airports_final
.repartition(1) # coalesce
.write.format("parquet")
.mode('overwrite')
.option("header", "true")
.save("./datasets/aviao/transformacao_airports_qa.parquet"))

In [125]:
(df_planes_final
.repartition(1) # coalesce
.write.format("parquet")
.mode('overwrite')
.option("header", "true")
.save("./datasets/aviao/transformacao_planes_qa.parquet"))

In [99]:
(df_flights
.repartition(1) # coalesce
.write.format("parquet")
.mode('overwrite')
.option("header", "true")
.save("./datasets/aviao/transformacao_flights_qa.parquet"))